In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('data/PremierLeague.csv')
df = df.drop(labels=[
    'Time', 'HomeTeamCorners', 'AwayTeamCorners', 'HalfTimeHomeTeamGoals', 'HalfTimeAwayTeamGoals', 'HalfTimeResult', 'HomeTeamShots', 'AwayTeamShots', 'HomeTeamShotsOnTarget', 'AwayTeamShotsOnTarget', 'B365HomeTeam', 'B365AwayTeam', 'B365Draw', 'B365Over2.5Goals', 'B365Under2.5Goals', 'MarketMaxHomeTeam', 'MarketMaxDraw', 'MarketMaxAwayTeam', 'MarketAvgHomeTeam', 'MarketAvgDraw', 'MarketAvgAwayTeam', 'MarketMaxOver2.5Goals', 'MarketMaxUnder2.5Goals', 'MarketAvgOver2.5Goals', 'MarketAvgUnder2.5Goals', 'HomeTeamPoints', 'AwayTeamPoints'
    ],
    axis = 1,
    )
df = df.dropna()
df.head()

,MatchID,Season,MatchWeek,Date,HomeTeam,AwayTeam,FullTimeHomeTeamGoals,FullTimeAwayTeamGoals,FullTimeResult,Referee,HomeTeamFouls,AwayTeamFouls,HomeTeamYellowCards,AwayTeamYellowCards,HomeTeamRedCards,AwayTeamRedCards
2660,2000-2001_Charlton_Man City,2000-2001,1,2000-08-19,Charlton,Man City,4,0,H,Rob Harris,13.0,12.0,1.0,2.0,0.0,0.0
2661,2000-2001_Chelsea_West Ham,2000-2001,1,2000-08-19,Chelsea,West Ham,4,2,H,Graham Barber,19.0,14.0,1.0,2.0,0.0,0.0
2662,2000-2001_Coventry_Middlesbrough,2000-2001,1,2000-08-19,Coventry,Middlesbrough,1,3,A,Barry Knight,15.0,21.0,5.0,3.0,1.0,0.0
2663,2000-2001_Derby_Southampton,2000-2001,1,2000-08-19,Derby,Southampton,2,2,D,Andy D'Urso,11.0,13.0,1.0,1.0,0.0,0.0
2664,2000-2001_Leeds_Everton,2000-2001,1,2000-08-19,Leeds,Everton,2,0,H,Dermot Gallagher,21.0,20.0,1.0,3.0,0.0,0.0


In [10]:
referee_stats = df.groupby('Referee')
referee_stats.head()

,MatchID,Season,MatchWeek,Date,HomeTeam,AwayTeam,FullTimeHomeTeamGoals,FullTimeAwayTeamGoals,FullTimeResult,Referee,HomeTeamFouls,AwayTeamFouls,HomeTeamYellowCards,AwayTeamYellowCards,HomeTeamRedCards,AwayTeamRedCards
2660,2000-2001_Charlton_Man City,2000-2001,1,2000-08-19,Charlton,Man City,4,0,H,Rob Harris,13.0,12.0,1.0,2.0,0.0,0.0
2661,2000-2001_Chelsea_West Ham,2000-2001,1,2000-08-19,Chelsea,West Ham,4,2,H,Graham Barber,19.0,14.0,1.0,2.0,0.0,0.0
2662,2000-2001_Coventry_Middlesbrough,2000-2001,1,2000-08-19,Coventry,Middlesbrough,1,3,A,Barry Knight,15.0,21.0,5.0,3.0,1.0,0.0
2663,2000-2001_Derby_Southampton,2000-2001,1,2000-08-19,Derby,Southampton,2,2,D,Andy D'Urso,11.0,13.0,1.0,1.0,0.0,0.0
2664,2000-2001_Leeds_Everton,2000-2001,1,2000-08-19,Leeds,Everton,2,0,H,Dermot Gallagher,21.0,20.0,1.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11758,2023-2024_Bournemouth_Brentford,2023-2024,37,2024-05-11,Bournemouth,Brentford,1,2,A,M Donohue,16.0,12.0,2.0,2.0,0.0,0.0
11777,2023-2024_Luton_Fulham,2023-2024,38,2024-05-19,Luton,Fulham,2,4,A,M Donohue,15.0,20.0,5.0,4.0,0.0,0.0
11803,2024-2025_Ipswich_Fulham,2024-2025,3,2024-08-31,Ipswich,Fulham,1,1,D,L Smith,15.0,15.0,2.0,3.0,0.0,0.0
11825,2024-2025_Southampton_Ipswich,2024-2025,5,2024-09-21,Southampton,Ipswich,1,1,D,S Allison,16.0,16.0,4.0,4.0,0.0,0.0
